__HOMEWORK WEEK 5__

Full name: Nguyễn Thị Minh Ngọc
<br>
Class: DSEB 63
<br>
Student ID: 11219280

In [1]:
import pandas as pd
import numpy as np

# Cleaning Data

In [3]:
df = pd.read_csv("../data/OnlineRetail2.csv")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,539993,22386,JUMBO BAG PINK POLKADOT,10,2011-01-04 10:00:00,1.95,13313.0,United Kingdom
1,539993,21499,BLUE POLKADOT WRAP,25,2011-01-04 10:00:00,0.42,13313.0,United Kingdom
2,539993,21498,RED RETROSPOT WRAP,25,2011-01-04 10:00:00,0.42,13313.0,United Kingdom
3,539993,22379,RECYCLING BAG RETROSPOT,5,2011-01-04 10:00:00,2.10,13313.0,United Kingdom
4,539993,20718,RED RETROSPOT SHOPPER BAG,10,2011-01-04 10:00:00,1.25,13313.0,United Kingdom


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203422 entries, 0 to 203421
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    203422 non-null  object 
 1   StockCode    203422 non-null  object 
 2   Description  202623 non-null  object 
 3   Quantity     203422 non-null  int64  
 4   InvoiceDate  203422 non-null  object 
 5   UnitPrice    203422 non-null  float64
 6   CustomerID   150039 non-null  float64
 7   Country      203422 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 12.4+ MB


__Check duplicated rows__

In [5]:
print("The number of the duplicated rows:", df.duplicated().sum())

The number of the duplicated rows: 1512


__Which attribute determines other attributes?__
<br>
&rarr; `StockCode` determines the `Description` of the products

__Check the distribution of the__ `Quantity` __attribute__

In [6]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,203422.000000,203422.000000,150039.000000
mean,9.585684,4.877820,15240.234832
std,240.921315,113.686676,1717.658493
min,-74215.000000,0.000000,12346.000000
25%,1.000000,1.250000,13791.000000
50%,3.000000,2.100000,15061.000000
75%,10.000000,4.130000,16771.000000
max,74215.000000,38970.000000,18287.000000


__Anomalies can be observed from the result above__
* The min value for `Quantity` is `-74215.0`, which means the `Quantity` attribute currently contains negative values.

__The percentage of null value for each attribute__

In [7]:
df.apply(lambda x: round(x.isnull().sum()/df.shape[0]*100, 2))

InvoiceNo       0.00
StockCode       0.00
Description     0.39
Quantity        0.00
InvoiceDate     0.00
UnitPrice       0.00
CustomerID     26.24
Country         0.00
dtype: float64

__Cleaning Data__

In [8]:
# Drop Duplicated Rows
df.drop_duplicates(inplace=True)
df.drop_duplicates(subset=['InvoiceNo', 'StockCode', 'Quantity'], inplace=True)

# Delete records with the value of Quantity less than 0
df = df[df['Quantity'] >= 0]

# Drop records with null CustomerID
df = df[df['CustomerID'].notnull()]

# Reformatting Data Type
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# RFM Analysis

In [9]:
customer_df = pd.DataFrame()

In [10]:
customer_df['CustomerID'] = df['CustomerID'].unique()
customer_df

,CustomerID
0,13313.0
1,18097.0
2,16656.0
3,16875.0
4,13094.0
...,...
2719,15578.0
2720,15584.0
2721,13745.0
2722,16498.0


__Calculating RFM Value__

In [11]:
# Frequency Value
frequency_df = df.groupby("CustomerID").nunique()['InvoiceNo'].reset_index().rename(columns={'InvoiceNo': 'Frequency'})
frequency_df

,CustomerID,Frequency
0,12346.0,1
1,12347.0,3
2,12348.0,2
3,12350.0,1
4,12352.0,5
...,...,...
2719,18273.0,1
2720,18280.0,1
2721,18281.0,1
2722,18283.0,7


In [12]:
# Monetary Value
df['Total Revenue'] = df['Quantity'] * df['UnitPrice']
monetary_df = df.groupby("CustomerID")['Total Revenue'].sum().reset_index().rename(columns={'Total Revenue': 'Monetary'})
monetary_df

,CustomerID,Monetary
0,12346.0,77183.60
1,12347.0,1494.16
2,12348.0,594.44
3,12350.0,334.40
4,12352.0,1098.01
...,...,...
2719,18273.0,51.00
2720,18280.0,180.60
2721,18281.0,80.82
2722,18283.0,812.04


In [13]:
# Requency Value
lastday = max(df["InvoiceDate"])
recency_df = df.groupby('CustomerID')['InvoiceDate'].max().reset_index().rename(columns={'InvoiceDate': 'Recency'})
recency_df['Recency'] = recency_df['Recency'].apply(lambda x: (lastday-x).days)
recency_df

,CustomerID,Recency
0,12346.0,163
1,12347.0,21
2,12348.0,86
3,12350.0,148
4,12352.0,100
...,...,...
2719,18273.0,95
2720,18280.0,115
2721,18281.0,18
2722,18283.0,7


In [14]:
customer_df = pd.merge(frequency_df, customer_df, on="CustomerID")
customer_df = pd.merge(customer_df, monetary_df, on="CustomerID")
customer_df = pd.merge(customer_df, recency_df, on="CustomerID")

In [15]:
customer_df

,CustomerID,Frequency,Monetary,Recency
0,12346.0,1,77183.60,163
1,12347.0,3,1494.16,21
2,12348.0,2,594.44,86
3,12350.0,1,334.40,148
4,12352.0,5,1098.01,100
...,...,...,...,...
2719,18273.0,1,51.00,95
2720,18280.0,1,180.60,115
2721,18281.0,1,80.82,18
2722,18283.0,7,812.04,7


__Add segment bin values to RFM table__

In [16]:
customer_df['r_quantile'] = 4 - pd.qcut(customer_df['Recency'], 4, labels=False, duplicates='drop')
customer_df['f_quantile'] = pd.qcut(customer_df['Frequency'], 4, labels=False, duplicates='drop') + 1
customer_df['m_quantile'] = pd.qcut(customer_df['Monetary'], 4, labels=False, duplicates='drop') + 1

__Calculating RFM Score__

In [17]:
customer_df['RFM_Score'] = customer_df['r_quantile'] + customer_df['f_quantile'] + customer_df['m_quantile']
customer_df

,CustomerID,Frequency,Monetary,Recency,r_quantile,f_quantile,m_quantile,RFM_Score
0,12346.0,1,77183.60,163,1,1,4,6
1,12347.0,3,1494.16,21,3,2,4,9
2,12348.0,2,594.44,86,2,1,3,6
3,12350.0,1,334.40,148,1,1,2,4
4,12352.0,5,1098.01,100,1,3,4,8
...,...,...,...,...,...,...,...,...
2719,18273.0,1,51.00,95,1,1,1,3
2720,18280.0,1,180.60,115,1,1,1,3
2721,18281.0,1,80.82,18,4,1,1,6
2722,18283.0,7,812.04,7,4,3,3,10


__Dividing customers into 3 segments (low-value, mid-value, high-value) based on__ `RFM_Score`

In [18]:
customer_df.sort_values(by='RFM_Score', ascending=False, inplace=True, ignore_index=True)
customer_df['Segmentation'] = pd.qcut(customer_df['RFM_Score'], q=[0, 0.5, 0.8, 1], labels=['low', 'mid', 'high'])

In [19]:
customer_df

,CustomerID,Frequency,Monetary,Recency,r_quantile,f_quantile,m_quantile,RFM_Score,Segmentation
0,15379.0,6,2211.83,7,4,3,4,11,high
1,17675.0,13,7765.30,0,4,3,4,11,high
2,16422.0,28,20197.22,2,4,3,4,11,high
3,13047.0,4,1867.45,11,4,3,4,11,high
4,14364.0,6,2808.00,2,4,3,4,11,high
...,...,...,...,...,...,...,...,...,...
2719,17654.0,1,117.55,115,1,1,1,3,low
2720,14238.0,1,146.85,164,1,1,1,3,low
2721,13104.0,1,111.96,105,1,1,1,3,low
2722,17639.0,1,108.50,114,1,1,1,3,low


In [20]:
result = customer_df[['CustomerID', 'Segmentation']]
result

,CustomerID,Segmentation
0,15379.0,high
1,17675.0,high
2,16422.0,high
3,13047.0,high
4,14364.0,high
...,...,...
2719,17654.0,low
2720,14238.0,low
2721,13104.0,low
2722,17639.0,low


In [21]:
# Percentage of each segment
result_p = pd.DataFrame(customer_df.groupby('Segmentation')['CustomerID'].count()).rename(columns={'CustomerID': 'Quantity'})
result_p['Percentage'] = result_p['Quantity'].apply(lambda x: round(x/customer_df.shape[0]*100, 2))
result_p

C:\Users\minhn\AppData\Local\Temp\ipykernel_22180\1014264948.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  result_p = pd.DataFrame(customer_df.groupby('Segmentation')['CustomerID'].count()).rename(columns={'CustomerID': 'Quantity'})


,Quantity,Percentage
Segmentation,,
low,1516,55.65
mid,750,27.53
high,458,16.81
